In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os, time, math, json
with open('/Users/yawagipong/.secret/tmdb_api.json', 'r') as f:
          login = json.load(f)
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['title.ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2001.json',
 'title_basics.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'title.akas.csv.gz',
 'tmdb_results_combined.csv.gz']

In [2]:
tmdb.API_KEY = login['api-key']

In [5]:
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [6]:
basics = basics.drop(columns = ['originalTitle', 'isAdult', 'titleType'])
basics.head()

,tconst,primaryTitle,startYear,endYear,runtimeMinutes,genres
0,tt0035423,Kate & Leopold,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020,NaN,70,Drama
2,tt0069049,The Other Side of the Wind,2018,NaN,122,Drama
3,tt0088751,The Naked Monster,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,Crime and Punishment,2002,NaN,126,Drama
